In [1]:
import xarray as xr
import glob
from dask.diagnostics import ProgressBar

In [2]:
rootdir = '/local/data/bSOSE/'
niter = 'iter133NEW'
freq = '5day'
chunks = 1

### CARBON CONCENTRATION BUDGET ###
# Load all carbon tendencies and fluxes
filenames = 'bsose_i133_2013to2018_5day_*C.nc'
ds = xr.open_mfdataset(rootdir+niter+'/'+freq+'/'+filenames,chunks={'time':chunks})
# Load surface carbon flux
filenames = 'bsose_i133_2013to2018_5day_surfCO2flx.nc'
ds_surf = xr.open_mfdataset(rootdir+niter+'/'+freq+'/'+filenames,chunks={'time':chunks})
# Load carbon snapshots
filenames = 'bsose_i133_2013to2018_5daySnapShots_DIC.nc'
ds_csnaps = xr.open_dataset(rootdir+niter+'/'+freq+'/'+filenames,chunks={'time':chunks})
ds_csnaps = ds_csnaps.rename({'time':'time_snaps','TRAC01':'TRAC01_snaps'}).drop('iter')
# Specify shift of time axis for snapshots
ds_csnaps['time_snaps'].attrs['c_grid_axis_shift']=-0.5

### VOLUME BUDGET ###
# Load velocity data
filenames = 'bsose_i133_2013to2018_5day_*vel.nc'
ds_vel = xr.open_mfdataset(rootdir+niter+'/'+freq+'/'+filenames,chunks={'time':chunks})
# Load SSH
filenames = 'bsose_i133_2013to2018_5day_SSH.nc'
ds_ssh = xr.open_mfdataset(rootdir+niter+'/'+freq+'/'+filenames,chunks={'time':chunks})
# Load SSH snapshots
filenames = 'bsose_i133_2013to2018_5daySnaps_SSH.nc'
ds_sshsnaps = xr.open_mfdataset(rootdir+niter+'/'+freq+'/'+filenames,chunks={'time':chunks})
ds_sshsnaps = ds_sshsnaps.rename({'time':'time_snaps','ETAN':'ETAN_snaps'}).drop('iter')
# Specify shift of time axis
ds_sshsnaps['time_snaps'].attrs['c_grid_axis_shift']=-0.5
# Load FW flux
freq = '1day'
filenames = 'bsose_i133_2013to2018_1dy_oceFWflx.nc'
ds_fw = xr.open_mfdataset(rootdir+niter+'/'+freq+'/'+filenames,chunks={'time':chunks})
ds_fw = ds_fw.coarsen(time=5,boundary='trim',keep_attrs=True).mean().assign_coords({'time':ds_vel['time']})

# Define vertical metrics as negative, to account for descending coordinate
ds['drW'] = ds.hFacW * ds.drF #vertical cell size at u point
ds['drS'] = ds.hFacS * ds.drF #vertical cell size at v point
ds['drC'] = ds.hFacC * ds.drF #vertical cell size at tracer point
# Volume
ds['vC'] = ds['drC']*ds['rA']
# Define cell side areas
ds['rAW'] = ds['dyG']*ds['drW']
ds['rAS'] = ds['dxG']*ds['drS']

# Merge to full dataset
ds = xr.merge([ds,ds_csnaps,ds_vel,ds_surf,ds_ssh,ds_sshsnaps,ds_fw])

In [3]:
ds.nbytes/1E9

1631.519399528

In [4]:
ds

<xarray.Dataset>
Dimensions:       (XC: 2160, XG: 2160, YC: 588, YG: 588, Z: 52, Zl: 52, time: 438, time_snaps: 439)
Coordinates: (12/21)
    iter          (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * time          (time) datetime64[ns] 2013-01-05 2013-01-10 ... 2018-12-30
  * XC            (XC) float32 0.08333 0.25 0.4167 0.5833 ... 359.6 359.8 359.9
  * YC            (YC) float32 -77.98 -77.95 -77.91 ... -30.02 -29.87 -29.72
  * Zl            (Zl) float32 0.0 -4.2 -9.2 ... -4.8e+03 -5.2e+03 -5.6e+03
    Depth         (YC, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    ...            ...
    rAs           (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    dxG           (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    dyC           (YG, XC) float32 dask.array<chunksize=(588, 2160), meta=np.ndarray>
    hFacS         (Z, YG, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
    hFacC         (Z, YC, XC) float32 dask.array<chunksize=(52, 588, 2160), meta=np.ndarray>
  * time_snaps    (time_snaps) datetime64[ns] 2012-12-31 ... 2018-12-30
Data variables: (12/24)
    ADVrTr01      (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    ADVxTr01      (time, Z, YC, XG) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    ADVyTr01      (time, Z, YG, XC) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    BLGBIOC       (time, Z, YC, XC) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    DFrITr01      (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    DFxETr01      (time, Z, YC, XG) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    ...            ...
    VVEL          (time, Z, YG, XC) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    WVEL          (time, Zl, YC, XC) float32 dask.array<chunksize=(1, 52, 588, 2160), meta=np.ndarray>
    BLGCFLX       (time, YC, XC) float32 dask.array<chunksize=(1, 588, 2160), meta=np.ndarray>
    ETAN          (time, YC, XC) float32 dask.array<chunksize=(1, 588, 2160), meta=np.ndarray>
    ETAN_snaps    (time_snaps, YC, XC) float32 dask.array<chunksize=(1, 588, 2160), meta=np.ndarray>
    oceFWflx      (time, YC, XC) float32 dask.array<chunksize=(1, 588, 2160), meta=np.ndarray>

In [7]:
with ProgressBar():
    ds.to_zarr('/local/data/bSOSE/iter133NEW/5day/zarr/bsose_i133_2013to2018_budgettermsDIC')

[####                                    ] | 10% Completed | 59.7s


KeyboardInterrupt: 

In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
